In [87]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from util import get_data, plot_data
import matplotlib.dates as mdates

In [44]:
def fill_missing_values(df_data):
    """Fill missing values in data frame, in place."""
    df_data.fillna(method="ffill", inplace=True)
    df_data.fillna(method="bfill", inplace=True)

def compute_daily_returns(df):
	"""Compute and return the daily return values."""
	daily_returns = df.copy()
	daily_returns[1:] = (df[1:] / df[:-1].values) - 1
	daily_returns.iloc[0] = 0 # set daily returns for row 0 to 0
	return daily_returns

def calculate_sharpe_ratio(daily_returns):
    """Calculate the Sharpe ratio for a given allocation and daily returns."""
    k = np.sqrt(252)  # Assuming 252 trading days in a year
    risk_free_rate = 0.0
    adr = daily_returns.mean()
    sddr = daily_returns.std()
    sr = k * (adr - risk_free_rate) / sddr
    return sr

def calculate_portfolio_value(allocs, prices):
    """Calculate the portfolio value for a given allocation and price data."""
    normed = prices / prices.iloc[0]
    alloced = normed * allocs
    pos_vals = alloced.sum(axis=1)
    return pos_vals

In [92]:
def optimize_portfolio(sd=dt.datetime(2008, 6, 1), ed=dt.datetime(2009, 6, 1), syms=['IBM', 'X', 'GLD', 'JPM'], gen_plot=False):
    # Read in adjusted closing prices for given symbols, date range
    dates = pd.date_range(sd, ed)
    prices_all = get_data(syms, dates)
    fill_missing_values(prices_all)
    prices = prices_all[syms]
    prices_SPY = prices_all['SPY']

    # Initial allocation guess (uniform allocation)
    n = len(syms)
    allocs = np.ones(n) / n

    # Calculate daily returns
    daily_returns = compute_daily_returns(prices)

    # Define the objective function for optimization (negative Sharpe ratio)
    def negative_sharpe_ratio(allocs, prices):
        pos_vals = calculate_portfolio_value(allocs, prices)
        port_returns = compute_daily_returns(pos_vals)
        sr = calculate_sharpe_ratio(port_returns)
        return -sr

    # Set optimization constraints
    bounds = [(0.0, 1.0) for _ in range(n)]
    constraints = ({'type': 'eq', 'fun': lambda allocs: np.sum(allocs) - 1.0})

    # Perform the optimization
    result = minimize(negative_sharpe_ratio, allocs, args=(prices,), method='SLSQP', bounds=bounds, constraints=constraints)
    allocs = np.round(result.x, decimals=6)

    # Calculate portfolio statistics using the optimal allocations
    pos_vals = calculate_portfolio_value(allocs, prices)
    port_returns = compute_daily_returns(pos_vals)
    cr = (pos_vals.iloc[-1] / pos_vals.iloc[0]) - 1
    adr = daily_returns.mean()
    sddr = daily_returns.std()
    sr = calculate_sharpe_ratio(port_returns)

    # Generate the plot if gen_plot is True
    if gen_plot:
        df_temp = pd.concat([pos_vals, prices_SPY], keys=['Portfolio', 'SPY'], axis=1)
        df_normed = df_temp / df_temp.iloc[0]
        ax = df_normed.plot(title='Daily Portfolio Value and SPY', fontsize=12)
        ax.set_ylabel('Price')
        ax.set_xlabel('Date')
        ax.legend(loc='upper left')
        
        # Set x-axis ticks to show each month
        ax.xaxis.set_major_locator(mdates.MonthLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        plt.savefig('./images/figure1.png')  # Save the plot to the "images" folder
        plt.close()  # Close the plot to prevent it from showing


    return allocs, cr, adr, sddr, sr


In [93]:
start_date = dt.datetime(2009, 1, 1)  		  	   		  		 			  		 			 	 	 		 		 	
end_date = dt.datetime(2010, 1, 1)  		  	   		  		 			  		 			 	 	 		 		 	
symbols = ["GOOG", "AAPL", "GLD", "XOM", "IBM"]  		  	   		  		 			  		 			 	 	 		 		 	
  		  	   		  		 			  		 			 	 	 		 		 	
# Assess the portfolio  		  	   		  		 			  		 			 	 	 		 		 	
allocations, cr, adr, sddr, sr = optimize_portfolio(  		  	   		  		 			  		 			 	 	 		 		 	
    sd=start_date, ed=end_date, syms=symbols, gen_plot=True  		  	   		  		 			  		 			 	 	 		 		 	
)
print(f"Start Date: {start_date}")  		  	   		  		 			  		 			 	 	 		 		 	
print(f"End Date: {end_date}")  		  	   		  		 			  		 			 	 	 		 		 	
print(f"Symbols: {symbols}")  		  	   		  		 			  		 			 	 	 		 		 	
print(f"Allocations:{allocations}")  		  	   		  		 			  		 			 	 	 		 		 	
print(f"Sharpe Ratio: {sr}")  		  	   		  		 			  		 			 	 	 		 		 	
print(f"Volatility (stdev of daily returns): {sddr}")  		  	   		  		 			  		 			 	 	 		 		 	
print(f"Average Daily Return: {adr}")  		  	   		  		 			  		 			 	 	 		 		 	
print(f"Cumulative Return: {cr}") 

Start Date: 2009-01-01 00:00:00
End Date: 2010-01-01 00:00:00
Symbols: ['GOOG', 'AAPL', 'GLD', 'XOM', 'IBM']
Allocations:[0.175123 0.401904 0.422973 0.       0.      ]
Sharpe Ratio: 3.011800611871923
Volatility (stdev of daily returns): GOOG    0.018852
AAPL    0.021036
GLD     0.013075
XOM     0.016333
IBM     0.017412
dtype: float64
Average Daily Return: GOOG    0.002788
AAPL    0.003568
GLD     0.000953
XOM    -0.000490
IBM     0.001833
dtype: float64
Cumulative Return: 0.7975537895375444


In [94]:
print(type(optimize_portfolio(sd=start_date, ed=end_date, syms=symbols, gen_plot=False))) 

<class 'tuple'>


In [46]:
start_val = 1000000
start_date = 2009-1-1
end_date = 2011-12-31
syms = ['SPY', 'XOM', 'GOOG', 'GLD']
allocs = [0.4, 0.4, 0.1, 0.1]
dates = pd.date_range(sd, ed)
prices_all = get_data(syms, dates)
fill_missing_values(prices_all)
prices = prices_all[syms]
prices_SPY = prices_all['SPY']
port_returns = calculate_portfolio_value(allocs, prices)
port_returns = compute_daily_returns(port_returns)
daily_returns = compute_daily_returns(prices_all)
print(port_returns)
print(daily_returns)
print(calculate_sharpe_ratio(daily_returns))

2009-01-02    0.000000
2009-01-05   -0.000450
2009-01-06   -0.001202
2009-01-07   -0.028628
2009-01-08    0.008890
                ...   
2009-12-24    0.007450
2009-12-28    0.004096
2009-12-29   -0.003618
2009-12-30   -0.000130
2009-12-31   -0.006962
Length: 252, dtype: float64
                 SPY       XOM      GOOG       GLD
2009-01-02  0.000000  0.000000  0.000000  0.000000
2009-01-05 -0.001152 -0.000134  0.020945 -0.020295
2009-01-06  0.006690 -0.016248  0.018320  0.007694
2009-01-07 -0.029904 -0.025526 -0.036071 -0.027957
2009-01-08  0.004016  0.010646  0.009875  0.020665
...              ...       ...       ...       ...
2009-12-24  0.004675  0.005965  0.011117  0.016987
2009-12-28  0.002140  0.006085  0.007098  0.001753
2009-12-29 -0.001393 -0.003412 -0.005571 -0.009949
2009-12-30 -0.000372 -0.001089  0.005376 -0.005025
2009-12-31 -0.009581 -0.008413 -0.004416  0.003554

[252 rows x 4 columns]
SPY     0.903706
XOM    -0.476652
GOOG    2.347641
GLD     1.157551
dtype: float64


In [95]:
    k = np.sqrt(252)  # Assuming 252 trading days in a year
    risk_free_rate = 0.0
    adr = 0.003
    sddr = 0.01
    sr = k * (adr - risk_free_rate) / sddr

In [96]:
print(sr)

4.762352359916264
